In [ ]:
import pandas as pd
import numpy as np
import os
import gzip
import pickle
import re
import copy
from tqdm import tqdm
import time

In [ ]:
hardware_instances = pd.read_csv('hardware_instances.csv') # columns: date,hardware

# # keep only values near the end of 2021
# hardware_instances = hardware_instances[hardware_instances["date"] > "2020-07-01"]
# hardware_instances = hardware_instances[hardware_instances["date"] < "2020-11-01"]

# unique_values = hardware_instances["hardware"].value_counts()
# pairs = []
# for i in range(0, len(unique_values)):
#     # print(unique_values.index[i], unique_values[i])
#     pairs.append((unique_values.index[i], unique_values[i]))

# pairs = sorted(pairs, key=lambda x: x[0], reverse=False)
# for pair in pairs:
#     # print(pair)
#     print(pair[0])

pd.set_option('display.max_rows', None)

# keep only values near the end of 2021
hardware_instances = hardware_instances[hardware_instances["date"] > "2020-07-01"]
hardware_instances = hardware_instances[hardware_instances["date"] < "2020-12-01"]
hardware_instances[["date", "hardware"]].to_csv("hardware_instances_2020_all.csv", index=False)

from hardware_mapping import map_hardware_to_table
hardware_instances = hardware_instances.assign(hardware_mapped = hardware_instances["hardware"].apply(map_hardware_to_table))
# hardware_instances

In [ ]:
hardware_instances = pd.read_csv('hardware_instances.csv') # columns: date,hardware
hardware_instances = hardware_instances[hardware_instances["date"] > "2018-01-01"]
hardware_instances = hardware_instances.assign(hardware_mapped = hardware_instances["hardware"].apply(map_hardware_to_table))

# find most common unique values that were mapped to "unknown"
unique_values = hardware_instances[hardware_instances["hardware_mapped"] == "unknown"]["hardware"].value_counts()
unique_values[:100]

In [ ]:
hardware_instances = pd.read_csv('hardware_instances.csv') # columns: date,hardware
hardware_instances = hardware_instances.assign(hardware_mapped = hardware_instances["hardware"].apply(lambda x: str(x).strip()).apply(map_hardware_to_table))

# print things that are "not found"
unique_values = hardware_instances[hardware_instances["hardware_mapped"] == "not found"]["hardware"].value_counts()
unique_values[:10]

In [ ]:
instances = pd.read_csv('hardware_instances_inc_threads.csv') # columns: date,thread


efficiency = pd.read_csv('hardware_instances_with_efficiency.csv') # columns: date,hardware_name,TH/J
efficiency["ln_efficiency"] = np.log(efficiency["TH/J"])
# efficiency = efficiency[efficiency["date"] > "2017-10-01"]
# efficiency = efficiency[efficiency["ln_efficiency"] < -9]
efficiency = efficiency[efficiency["date"] < "2013-10-01"]
efficiency = efficiency[efficiency["ln_efficiency"] > -9.5]

i = 0
for index, row in efficiency.iterrows():
    print(row["date"], row["hardware_name"], row["TH/J"], row["ln_efficiency"])
    filtered = instances[instances["date"] == row["date"]]
    print(filtered.iloc[0]["thread"])
    print("\n\n\n\n\n\n\n\n\n\n\n\n")
    if i > 5:
        break
    i += 1